In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math as math

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:

csv1_path = '[Put path to file #1 here.]'
csv2_path = '[Put path to file #2 here.]'
csv3_path = '[Put path to file #3 here.]'
csv4_path = '[Put path to file #4 here.]'
csv5_path = '[Put path to file #5 here.]'
csv6_path = '[Put path to file #6 here.]'

prop_df1 = pd.read_csv(csv1_path)
prop_df2 = pd.read_csv(csv2_path)
prop_df3 = pd.read_csv(csv3_path)
prop_df4 = pd.read_csv(csv4_path)
prop_df5 = pd.read_csv(csv5_path)
prop_df6 = pd.read_csv(csv6_path)

prop_df1.head(3)

In [ ]:
prop_df = pd.concat([prop_df1, prop_df2])
prop_df = pd.concat([prop_df, prop_df3])
prop_df = pd.concat([prop_df, prop_df4])
prop_df = pd.concat([prop_df, prop_df5])
prop_df = pd.concat([prop_df, prop_df6])
prop_df.shape

In [ ]:
prop_df.columns

In [ ]:
prop_df.rename(index=str, columns={'$/SQUARE FEET':'PRICE PER SQFT',
                                  'HOA/MONTH':'HOA PER MONTH',
                                  'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL',
                                  'MLS#':'MLS'}, inplace=True)
prop_df.columns
#set(prop_df['PROPERTY TYPE'])

In [ ]:
# explore distribution of numeric columns
ax_list = prop_df.hist(bins=25, layout=(4,4), figsize=(15,15))

In [ ]:
# drop rows with missing values in critical columns
prop_df_nona = prop_df.dropna(axis=0, how='any', # if any of these cols are empty, remove row
                              subset=['LATITUDE','LONGITUDE'])
                              #subset=['BEDS','BATHS', 'PRICE', 'LATITUDE','LONGITUDE','SQUARE FEET'])
prop_df_nona['HOA PER MONTH'].fillna(value=0, inplace=True)
prop_df_nona.shape
#set(prop_df_nona['PROPERTY TYPE'])

In [ ]:
# explore distribution of numeric columns
ax_list = prop_df_nona.hist(bins=25, layout=(4,4), figsize=(15,15))

In [ ]:
prop_df_nona.describe().round(3)

In [ ]:
prop_df_nona.to_csv('houses_for_sale_filtered.csv')

In [ ]:
filtered_df = prop_df_nona[(prop_df_nona['PRICE']<=200000) &
                       (prop_df_nona['LOT SIZE']>=20000) & 
                        ((prop_df_nona['PROPERTY TYPE'] == 'Vacant Land') | 
                         (prop_df_nona['PROPERTY TYPE'] == 'Single Family Residential') |
                         (prop_df_nona['PROPERTY TYPE'] == 'Mobile/Manufactured Home'))]
filtered_df.shape

In [ ]:
ax_list2 = filtered_df.hist(bins=25, layout=(4,4), figsize=(15,15))

In [ ]:
filtered_df.head(9)

In [ ]:
filtered_df.to_csv('houses_for_sale_selected.csv')

In [ ]:
pricemax = np.max(filtered_df['PRICE'])
pricemin = np.min(filtered_df['PRICE'])
priceoff = filtered_df['PRICE'] - pricemin
pricescore = 1 - priceoff/np.max(priceoff)
bedsscore = filtered_df['BEDS']/4
sqftscore = filtered_df['SQUARE FEET']/np.max(filtered_df['SQUARE FEET'])

lotmax = np.max(filtered_df['LOT SIZE'])
lotmin = np.min(filtered_df['LOT SIZE'])
lotscore = 2*filtered_df['LOT SIZE']/lotmax

Re = 6371000 #radius of the Earth
worklatt = 41.948463 #work latitude
worklong = -87.655800 #work longitude
churchlatt = 42.36643289381998 #church latitude
churchlong =  -71.0544051307879 #church longitude
oldhouselatt = 37.62844044850603 #old house latitude
oldhouselong = -85.63519131337189 #old house longitude

work_x_dist = (filtered_df['LONGITUDE']*np.pi/180 - worklong*np.pi/180)*np.cos((filtered_df['LATITUDE']-worklatt)*np.pi/360)
work_y_dist = (filtered_df['LATITUDE'] - worklatt)*np.pi/180
workdist = np.sqrt(work_x_dist*work_x_dist + work_y_dist*work_y_dist)*Re
workdistmax = np.max(workdist)
workdistmin = np.min(workdist)
workdistscore = 1-workdist/workdistmax

church_x_dist = (filtered_df['LONGITUDE']*np.pi/180 - churchlong*np.pi/180)*np.cos((filtered_df['LATITUDE']-churchlatt)*np.pi/360)
church_y_dist = (filtered_df['LATITUDE'] - churchlatt)*np.pi/180
churchdist = np.sqrt(church_x_dist*church_x_dist + church_y_dist*church_y_dist)*Re
churchdistmax = np.max(churchdist)
churchdistmin = np.min(churchdist)
churchdistscore = 1-churchdist/churchdistmax

oldhouse_x_dist = (filtered_df['LONGITUDE']*np.pi/180 - oldhouselong*np.pi/180)*np.cos((filtered_df['LATITUDE']-oldhouselatt)*np.pi/360)
oldhouse_y_dist = (filtered_df['LATITUDE'] - oldhouselatt)*np.pi/180
oldhousedist = np.sqrt(oldhouse_x_dist*oldhouse_x_dist + oldhouse_y_dist*oldhouse_y_dist)*Re
oldhousedistmax = np.max(oldhousedist)
oldhousedistmin = np.min(oldhousedist)
oldhousedistscore = 1-oldhousedist/oldhousedistmax

overalldist = np.array([[workdistscore],[churchdistscore],[oldhousedistscore]])
overalldistscore = pd.DataFrame(index=range(9),columns=range(0))
overalldistscore_trans = np.mean(overalldist,axis=0).T
overalldistscore = 4*overalldistscore_trans.reshape(-1)
#print(2*overalldistscore.reshape(-1))
#overalldistscore = 4*workdistscore

overallscore_dist = np.array([[pricescore],[lotscore],[overalldistscore]])
overallscore_trans = np.mean(overallscore_dist,axis=0).T
overallscore = overallscore_trans.reshape(-1)

In [ ]:
filtered_df.loc[:,'PRICE SCORE'] = pd.Series(pricescore, index=filtered_df.index)
filtered_df.loc[:,'BEDS SCORE'] = pd.Series(bedsscore, index=filtered_df.index)
#filtered_df.loc[:,'BATHS SCORE'] = pd.Series(bathscore, index=filtered_df.index)
filtered_df.loc[:,'SQ. FT. SCORE'] = pd.Series(sqftscore, index=filtered_df.index)
filtered_df.loc[:,'LOT SCORE'] = pd.Series(lotscore, index=filtered_df.index)
filtered_df.loc[:,'WORK DIST SCORE'] = pd.Series(workdistscore, index=filtered_df.index)
filtered_df.loc[:,'CHURCH DIST SCORE'] = pd.Series(churchdistscore, index=filtered_df.index)
filtered_df.loc[:,'OLD HOUSE DIST SCORE'] = pd.Series(oldhousedistscore, index=filtered_df.index)
#filtered_df.loc[:,'OVERALL DIST SCORE'] = pd.Series(overalldistscore, index=filtered_df.index)
filtered_df.loc[:,'OVERALL SCORE'] = pd.Series(overallscore, index=filtered_df.index)

In [ ]:
filtered_df.head(9)

In [ ]:
filtered_df.to_csv('houses_for_sale_scored.csv')